In [14]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('best_model_bert')
model = BertForSequenceClassification.from_pretrained('best_model_bert')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

emotion_labels = ["anger", "fear", "joy", "sadness", "surprise"]

text = "ayush is sad and slightly happy"

inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
inputs = {key: value.to(device) for key, value in inputs.items()}

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

probabilities = torch.sigmoid(logits)

predictions = (probabilities > 0.5).int()

predicted_emotions = [emotion_labels[i] for i in range(len(predictions[0])) if predictions[0][i] == 1]

if predicted_emotions:
    print("Predicted Emotions:", ", ".join(predicted_emotions))
else:
    print("No emotions predicted")


Predicted Emotions: joy, sadness


In [15]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import gradio as gr

# Load the tokenizer and the new intensity prediction model
tokenizer = BertTokenizer.from_pretrained('best_model_bert_intensity')
model = BertForSequenceClassification.from_pretrained('best_model_bert_intensity')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

emotion_labels = ["anger", "fear", "joy", "sadness", "surprise"]

def predict_emotions_intensity(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    probabilities = torch.sigmoid(logits).squeeze().cpu().numpy()
    emotion_intensity = [(emotion_labels[i], probabilities[i]) for i in range(len(probabilities))]
    emotion_intensity.sort(key=lambda x: x[1], reverse=True)
    intensity_predictions = [f"{emotion}: {intensity:.2f}" for emotion, intensity in emotion_intensity]

    return ", ".join(intensity_predictions)

interface = gr.Interface(
    fn=predict_emotions_intensity,
    inputs="text",
    outputs="text",
    title="Emotion Intensity Detection",
    description="Enter text to get the intensity of emotions (0-1 scale)",
    examples=[["I am feeling happy"], ["This is a really sad day"], ["I'm so angry and frustrated"], ["I got a gift."]]
)

interface.launch()


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
